In [0]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam (1).csv


In [0]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [0]:
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
v1            5572 non-null object
v2            5572 non-null object
Unnamed: 2    50 non-null object
Unnamed: 3    12 non-null object
Unnamed: 4    6 non-null object
dtypes: object(5)
memory usage: 217.8+ KB


In [0]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [0]:
df.rename(columns = {'v2':'text messages',
                     'v1': 'label'})

In [0]:
def print_plot(index):
    example = df[df.index == index][['v2', 'v1']].values[0]
    if len(example) > 0:
        print(example[0])
        #print('Label:', example[1])


In [0]:
print_plot(5569)

pity mood soany suggestions


In [0]:
import nltk

In [0]:
import nltk
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['v2'] = df['v2'].apply(clean_text)

In [0]:
print_plot(8)

winner valued network customer selected receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hours


In [0]:
X = df['v2'].values
Y = df['v1'].values

le = LabelEncoder()
Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)


In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [0]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM,  Dense, Dropout, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential

max_words = 1000
max_len = 150
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
print(sequences)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

[[152, 5, 872], [307, 130, 18, 1], [14, 873, 974, 46, 974, 93, 463, 339, 67, 974, 975, 394, 244, 874, 31], [4, 567, 277], [68, 162, 142, 1, 1], [340], [110, 136, 439, 68], [93, 58, 49, 115, 96, 710, 5, 503, 289, 778, 48, 18, 51], [18, 83, 22, 12, 156, 146, 58, 20, 416, 417, 188, 88, 174, 94, 22, 245], [138, 652, 156, 50, 87, 69], [189, 36], [90, 225], [440, 15], [1, 43, 257, 1, 40, 6, 234, 568], [17, 55, 226], [40, 139, 3, 976, 3, 131, 21, 68, 49, 48, 116, 91, 1], [114], [42, 611, 4, 124], [23, 7, 10, 125], [27, 166, 359], [612, 52], [18, 20], [], [258, 325, 613], [464, 464, 464, 18, 653, 875], [35, 19], [977, 130, 569, 465, 978, 290, 441], [29, 122, 157, 227], [1, 235, 63], [5, 246, 876, 11, 876, 504, 570, 31], [418, 533, 126], [5, 54], [534, 3, 614, 1, 45, 11, 47, 2, 37, 37, 47, 1], [163, 49, 505], [47, 90, 466, 158], [1, 535, 711, 278, 3, 66, 103, 877, 878, 536, 6, 879], [4, 712, 7, 10, 341, 395, 136, 147, 654, 979, 202, 8, 47], [308, 18, 38, 1, 340], [228, 30, 209, 419, 132, 1, 12,

In [0]:
model = Sequential()
model.add(Embedding(max_words, 50, input_length=max_len))
model.add(LSTM(64))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [0]:
from tensorflow.keras.callbacks import EarlyStopping

model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 3565 samples, validate on 892 samples
Epoch 1/10
3565/3565 [==============================] - 10s 3ms/sample - loss: 0.3618 - acc: 0.8656 - val_loss: 0.2087 - val_acc: 0.8879
Epoch 2/10
3565/3565 [==============================] - 9s 2ms/sample - loss: 0.1726 - acc: 0.9447 - val_loss: 0.1147 - val_acc: 0.9652
Epoch 3/10
3565/3565 [==============================] - 9s 3ms/sample - loss: 0.0921 - acc: 0.9767 - val_loss: 0.0710 - val_acc: 0.9854
Epoch 4/10
3565/3565 [==============================] - 9s 3ms/sample - loss: 0.0635 - acc: 0.9829 - val_loss: 0.0529 - val_acc: 0.9865
Epoch 5/10
3565/3565 [==============================] - 9s 3ms/sample - loss: 0.0458 - acc: 0.9871 - val_loss: 0.0462 - val_acc: 0.9899
Epoch 6/10
3565/3565 [==============================] - 9s 2ms/sample - loss: 0.0398 - acc: 0.9893 - val_loss: 0.0455 - val_acc: 0.9899
Epoch 7/10
3565/3565 [==============================] - 9s 3ms/sample - loss: 0.0316 - acc: 0.9913 - val_loss: 0.0473 - val_acc: 0.9888


In [0]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [0]:
accr = model.evaluate(test_sequences_matrix,Y_test)

1115/1115 [==============================] - 1s 945us/sample - loss: 0.0446 - acc: 0.9874
